In [1]:
import openai
import langchain
from dotenv import load_dotenv,find_dotenv

from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SimpleSequentialChain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.agents.agent_toolkits import create_python_agent
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
#from langchain.llms.openai import OpenAI
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import JSONLoader
from langchain.document_loaders import Docx2txtLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

import json
from pathlib import Path
from pprint import pprint
import pinecone
from tqdm.notebook import tqdm # need this to stop warning

import os
import pinecone # pip3 install "pinecone-client[grpc]"
from langchain.vectorstores import Pinecone

from pprint import pprint # pretty print

# load environmnet variables
load_dotenv('.env')

C:\Users\vthammuneni\Anaconda3\envs\tf\lib\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


True

In [2]:
### Load only the syllabus summaerized text files

In [33]:
directory = './check/'

def load_docs(directory):
  loader = DirectoryLoader(directory,silent_errors=True)
  documents = loader.load()
  return documents

documents_text = load_docs(directory)
len(documents_text)

Error loading file check\~$23 Spring_ISM 6251-002-Data Science Programming_Smith, Tim.docx: Package not found at 'check\~$23 Spring_ISM 6251-002-Data Science Programming_Smith, Tim.docx'


3

In [34]:
def split_docs(documents,chunk_size=300,chunk_overlap=100):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  docs = text_splitter.split_documents(documents)
  return docs

In [35]:
docs_docx = split_docs(documents_text)

In [36]:
docs_docx[0].page_content

'Each module contains lecture slides, assignments, quizzes, and other materials. Professor: Dr. Tim Smith Course : ISM 6251-002-Data Science Programming.Course Topics: Data analytics techniques, tools and applications, machine learning techniques for analytics using Python, decision trees, gradient'

In [17]:
len(docs_docx)

9097

In [18]:
## embedding
embeddings = OpenAIEmbeddings(deployment="text-embedding-ada-002")

In [19]:
### Loading in to pinecone:

In [20]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_INDEX_NAME = os.getenv("PINECONE_INDEX_NAME")
PINECONE_ENVIRONMENT = os.getenv("PINECONE_ENV")

In [21]:
# create an index
# NOTE: For free version, you can only create one index. If you want to create a new index, you need to delete the old one first.

pinecone.init(api_key=PINECONE_API_KEY, environment=PINECONE_ENVIRONMENT)

#pinecone.delete_index("chatbot")

pinecone.create_index("chatbot", dimension=1536) # 1536 is openai ada embedding dimension

In [22]:
index_name = "chatbot"

In [23]:
search = Pinecone.from_documents(docs_docx, embeddings, index_name=index_name)

In [24]:
### Documents from the web

In [25]:
directory = './All_Texts/'

def load_docs(directory):
  loader = DirectoryLoader(directory,silent_errors=True)
  documents = loader.load()
  return documents

documents_text = load_docs(directory)
len(documents_text)

21

In [26]:
def split_docs(documents,chunk_size=315,chunk_overlap=100):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  docs = text_splitter.split_documents(documents)
  return docs

In [27]:
docs_docx_1 = split_docs(documents_text)

In [28]:
len(docs_docx_1)

1021

In [29]:
search = Pinecone.from_documents(docs_docx_1, embeddings, index_name=index_name)

In [30]:
############################### Checking the accuracy

In [31]:
from langchain.vectorstores import Pinecone
text_field = "text"

# switch back to normal index for langchain
index = pinecone.Index(index_name)

In [32]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [33]:
from langchain.embeddings.openai import OpenAIEmbeddings

model_name = 'text-embedding-ada-002'

embed = OpenAIEmbeddings(
    model=model_name,
    openai_api_key=OPENAI_API_KEY
)

In [34]:
vectorstore = Pinecone(
    index, embed.embed_query, text_field
)

C:\Users\vthammuneni\Anaconda3\envs\tf\lib\site-packages\langchain\vectorstores\pinecone.py:59: UserWarning: Passing in `embedding` as a Callable is deprecated. Please pass in an Embeddings object instead.
  warnings.warn(


In [35]:
query = "What is the appplication deadline for fall?"

vectorstore.similarity_search_with_score(query, k=2)

[(Document(page_content='get visa appointments at their local U.S. consulate or embassy. For the Spring semester, our application deadline is August 1 for international students and November 1 for domestic students. However, we admit most applicants in the Fall, and the number', metadata={'source': 'All_Texts\\Copy of usfmsbais.txt'}),
  0.851526737),
 (Document(page_content='Drop/Add Deadline (Aug 25, 2023), Mid-term Grading Closes (Oct 17, 2023), Withdrawal Deadline (Oct 28, 2023), Test Free Week (Apr 22-28, 2023), and Final Examination Week (Dec 2-7, 2023).', metadata={'source': 'pdf_word\\QMB3200.902F23 & QMB3200.521F23 - Govindu.docx'}),
  0.836478114)]

In [36]:
query = "What are the core courses for the bais program"

vectorstore.similarity_search_with_score(query, k=3)

[(Document(page_content='What are the different core and elective course options that I have for my MS-BAIS degree?\n\nMS\n\n\n\nBAIS core classes:', metadata={'source': 'All_Texts\\Copy of MS-BAIS Advising FAQ.txt'}),
  0.888100088),
 (Document(page_content='The program requires a minimum of 33 credits, which must include 5 core classes, 3 MS-BAIS electives and 3 additional electives from our department or other departments if they are 6000 level courses relevant to the major. Most of these classes are 3 credits each. In addition, you can also take MS-BAIS', metadata={'source': 'All_Texts\\Copy of MS-BAIS Advising FAQ.txt'}),
  0.871266961),
 (Document(page_content='How do I graduate from the BAIS program?', metadata={'source': 'All_Texts\\Copy of MS-BAIS Advising FAQ.txt'}),
  0.863762617)]

In [37]:
query = "What are the oncampus housing options?"

vectorstore.similarity_search_with_score(query, k=2)

[(Document(page_content='Show Housing Options\n\nOverview\n\nApartment Style(https://www.usf.edu/housing/housing-options/apartment-style/index.aspx)\n\nSuite Style(https://www.usf.edu/housing/housing-options/suite-style/index.aspx)\n\nTraditional Style(https://www.usf.edu/housing/housing-options/traditional-style/index.aspx)', metadata={'source': 'All_Texts\\usf_on_campus_housing.txt'}),
  0.85402596),
 (Document(page_content='The on-campus option is designed for students who need flexibility in their course work. Students will work with faculty to design the most effective course sequence and optional thesis/practicum /independent studies to meet the major curriculum requirements and accomplish their career goals.', metadata={'source': 'All_Texts\\Copy of usfmsbais.txt'}),
  0.85029459)]

In [38]:
query = "Project management course syllabus "

vectorstore.similarity_search_with_score(query, k=2)

[(Document(page_content='Course Topics: Introduction to Project, Program, and Portfolio Management, Agile Mindset, Project Management in Practice, Project Management Software, Differences between Predictive Agile, and Hybrid Project Management, Project Scope & Work Breakdown Structure (WBS), Project Scheduling & Gantt', metadata={'source': 'pdf_word\\ISM 6316 - McNair.docx'}),
  0.894224763),
 (Document(page_content='This is a course on Project Management with Agility taught by Professor Bhuvan Unhelkar. The topics covered include Agile Methodologies, Planning, Estimation, Risk Management, Quality Management, Change Management, Monitoring and Control, Team Management and Leadership. Assessments include', metadata={'source': 'pdf_word\\2023 Spring_ISM 4314-521_Project Management_Unhelkar, Bhuvan.docx'}),
  0.894158423)]

In [39]:
query = "what does professor Tim Smith teach "
vectorstore.similarity_search_with_score(query, k=3)

[(Document(page_content='Professor: Dr. John Smith', metadata={'source': 'pdf_word\\2023 Spring_ISM 4212- 020_Database Administration and Design_ Hyman_Harvey  .docx'}),
  0.880958796),
 (Document(page_content='Each module contains lecture slides, assignments, quizzes, and other materials. Professor: Dr. Tim Smith', metadata={'source': 'pdf_word\\cleaned_2023 Spring_ISM 6251-002-Data Science Programming_Smith, Tim.docx'}),
  0.879039228),
 (Document(page_content='Each module contains lecture slides, \nassignments, quizzes, and other materials. \n \nProfessor: Dr. Tim Smith', metadata={'source': 'pdf_word\\2023 Spring_ISM 6251-002-Data Science Programming_Smith, Tim.docx'}),
  0.872119069)]

In [40]:
query = "Business realted prerequisities for the MS BAIS program"

vectorstore.similarity_search_with_score(query, k=3)

[(Document(page_content='In addition, certain core MS-BAIS classes like Advanced Database Management have pre-program prerequisites such as a database prerequisite, which can be met by virtue of a prior course in database management from your previous university or adequate working experience with databases in a corporate environment.', metadata={'source': 'All_Texts\\Copy of Course Registration.txt'}),
  0.891857147),
 (Document(page_content='Admission Information\n\nMust meet University Admission and English Proficiency requirements , as well as requirements for admission to the major, listed below. Students are admitted to the M.S./BAIS program based on the evaluation of their application in its entirety, including:', metadata={'source': 'All_Texts\\Copy of usfmsbais.txt'}),
  0.889007628),
 (Document(page_content='and Information Systems (MS-BAIS) program. The program is designed to provide students with the knowledge and skills to become successful business analytics and informati